In [1]:
import pandas as pd
import numpy as np

In [2]:
from pandas import read_csv
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math
import matplotlib.pyplot as plt

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
df = pd.read_csv('/content/drive/MyDrive/audio-interspeech/2075_concatenate_dm_4featu.csv')
df = df.drop(['Unnamed: 0'], axis = 1)
df.shape

(2075, 181)

In [5]:
np.random.seed(42)
siz=415
df_read = df.copy()
df1 = df_read.sample(siz)
df_read = df_read.drop(df1.index)
df2 = df_read.sample(siz)
df_read = df_read.drop(df2.index)
df3 = df_read.sample(siz)
df_read = df_read.drop(df3.index)
df4 = df_read.sample(siz)
df_read = df_read.drop(df4.index)
df5 = df_read.copy()

print(df1.shape)
print(df2.shape)
print(df3.shape)
print(df4.shape)
print(df5.shape)

(415, 181)
(415, 181)
(415, 181)
(415, 181)
(415, 181)


In [6]:
q = list(df1.index)+list(df2.index)+list(df3.index)+list(df4.index)+list(df5.index)
print(df1.index)

Int64Index([ 881,  453, 2004, 1353,  281,  941, 1185, 1159, 1138,  599,
            ...
             834, 1730,  353, 1345, 1190, 1375,  185,  701, 1671, 1982],
           dtype='int64', length=415)


In [7]:
train = pd.concat([df1,df2,df3,df4], axis=0)
test = df5.copy()

Y_train = np.array(train['class'])
X_train= np.array(train.drop(['class'],axis=1))
# X_train=X_train.reshape(X_train.shape[0], 1 , X_train.shape[1])

Y_val=np.array(test['class'])
X_val = np.array(test.drop(['class'],axis=1))
# X_val=X_val.reshape(X_val.shape[0], 1 , X_val.shape[1])

In [8]:
TIME_STEPS= X_train.shape[1]

In [9]:
def create_LSTM(hidden_units, dense_units, input_shape, activation):
    model = Sequential()
    model.add(LSTM(hidden_units,input_shape=input_shape,return_sequences=False))
    model.add(Dense(units = input_shape[0]//2, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units=input_shape[0]//4, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units=1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [10]:
model = create_LSTM(hidden_units=180, dense_units=1, input_shape=(TIME_STEPS,1),
                   activation=['tanh', 'tanh'])
model.fit(X_train, Y_train, epochs=500, batch_size=64, verbose=2,validation_data=(X_val, Y_val))

Epoch 1/500
26/26 - 9s - loss: 5.7776 - val_loss: 1.8387 - 9s/epoch - 348ms/step
Epoch 2/500
26/26 - 0s - loss: 3.0517 - val_loss: 1.7770 - 313ms/epoch - 12ms/step
Epoch 3/500
26/26 - 0s - loss: 2.8043 - val_loss: 1.7717 - 345ms/epoch - 13ms/step
Epoch 4/500
26/26 - 0s - loss: 2.6102 - val_loss: 1.6023 - 313ms/epoch - 12ms/step
Epoch 5/500
26/26 - 0s - loss: 2.5490 - val_loss: 1.8804 - 374ms/epoch - 14ms/step
Epoch 6/500
26/26 - 0s - loss: 2.5483 - val_loss: 1.6438 - 345ms/epoch - 13ms/step
Epoch 7/500
26/26 - 0s - loss: 2.5446 - val_loss: 1.7266 - 314ms/epoch - 12ms/step
Epoch 8/500
26/26 - 0s - loss: 2.4411 - val_loss: 1.6272 - 314ms/epoch - 12ms/step
Epoch 9/500
26/26 - 0s - loss: 2.3644 - val_loss: 1.7302 - 322ms/epoch - 12ms/step
Epoch 10/500
26/26 - 0s - loss: 2.3271 - val_loss: 1.5188 - 319ms/epoch - 12ms/step
Epoch 11/500
26/26 - 0s - loss: 2.2229 - val_loss: 1.4746 - 344ms/epoch - 13ms/step
Epoch 12/500
26/26 - 0s - loss: 2.0370 - val_loss: 1.5633 - 315ms/epoch - 12ms/step
Epo

In [11]:
#run
import time
st = time.time()
p1 = np.array(model(X_val)).flatten()
end = time.time()
# print(end, st, len(p1))
print((end-st)/len(p1))
p2 = np.array(model(X_train)).flatten()
print(X_train.shape)
print(X_val.shape)

0.0001585282475115305
(1660, 180)
(415, 180)


In [12]:
from scipy import stats
from scipy.stats import kendalltau
# print(np.corrcoef(p2, Y_train), stats.spearmanr(p2, Y_train), kendalltau(p2,Y_train).correlation)
print(np.corrcoef(p1, Y_val)[1][0], stats.spearmanr(p1, Y_val).correlation, kendalltau(p1,Y_val).correlation)


0.7664906042662938 0.769670088570305 0.574385848441293
